In [3]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os

# Get mito genes location

We subsetted the hg38 reference genome GTF file to obtain the location of chrM genes.

In [22]:
# relative to repo
mito_genes_path = '../../misc/mito_genes.csv'

In [24]:
# Load mito genes
mito_genes = pd.read_csv(f'{mito_genes_path}', index_col=0)
mito_genes

,seqname,source,feature,start,end,score,strand,frame,gene_id,gene_version,...,transcript_type,transcript_name,transcript_support_level,havana_transcript,exon_number,exon_id,exon_version,protein_id,ccdsid,ont
2765275,chrM,ENSEMBL,gene,3307,4262,NaN,+,0,ENSG00000198888,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2765279,chrM,ENSEMBL,gene,4470,5511,NaN,+,0,ENSG00000198763,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2765283,chrM,ENSEMBL,gene,5904,7445,NaN,+,0,ENSG00000198804,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2765288,chrM,ENSEMBL,gene,7586,8269,NaN,+,0,ENSG00000198712,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2765295,chrM,ENSEMBL,gene,8366,8572,NaN,+,0,ENSG00000228253,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2765302,chrM,ENSEMBL,gene,8527,9207,NaN,+,0,ENSG00000198899,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2765309,chrM,ENSEMBL,gene,9207,9990,NaN,+,0,ENSG00000198938,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2765314,chrM,ENSEMBL,gene,10059,10404,NaN,+,0,ENSG00000198840,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2765318,chrM,ENSEMBL,gene,10470,10766,NaN,+,0,ENSG00000212907,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2765325,chrM,ENSEMBL,gene,10760,12137,NaN,+,0,ENSG00000198886,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 00 Getting ATAC Fragments

**NOTE:** We obtained our mito fragments from the fastq files by running Cellranger ARC for multiomic data. 

**However, we made the following changes to run Cellranger ARC:**
- Used a custom CellRanger ARC reference genome. We blacklisted nuclear regions that were homologous to the mitochondrial genome in our reference genome. We followed the steps here (https://github.com/caleblareau/mitoblacklist). This prevents mito reads from being multimapped and thrown out and increased the coverage of our mitochondrial reads.
- Move `chrM` from `non_nuclear_contigs` to `primary_contigs` in the CellRanger ARC configuration file. This ensures the mitochondrial reads are not discarded during processing.

After processing, CellRanger ARC provides us with `atac_fragments.tsv.gz` as output. This file contains information on all the ATAC fragments sequenced. This file is used as input for the next steps of processing:

# 01 Subset Multiome ATAC fragments to just Mito

The steps in this file are essentially the same processing steps as we did for the 8HetClonesHashtagMix samples. We just use a for loop to process our 3 samples (Glucose + Pyruvate, Glucose - Pyruvate, Galactose).

In [25]:
samples = [
    "YF-2721_48mix_25dGlu_3d_mito_hLLLdogma_multiome", # Glucose + Pyruvate
    "YF-2721_48mix_10dGal_3d_mito_hLLLdogma_multiome", # Galactose
    "YF-2721_48mix_25dGlu_NoPyr_3d_mito_hLLLdogma_multiome", # Glucose - Pyruvate
]

Our file structure may be different from yours, just point to the correct files. What you will need is:
- input path to `atac_fragments.tsv.gz` which is the output of Cellranger ARC.
- output path to store results

In [ ]:
# edit to your file structure
data_dir = '/data/peer/landm/Projects/sail-dogma/yi-clone-samples/data/'
cache_dir = '/data/peer/landm/Projects/sail-dogma/yi-clone-samples/cache/'

for s in samples:
    sample_dir = f"{data_dir}/{s}/cr-arc-results/"
    
    # READ INPUT
    fragments = pd.read_csv(f"{sample_dir}atac_fragments.tsv.gz", compression='gzip', header = None, sep='\t', quotechar='"', comment = '#')
    mito_fragments = fragments[fragments[0] == 'chrM']
    print(f'{mito_fragments.shape[0]} mitochondrial fragments')

    # Annotate fragments by where they fall on the genome
    mito_fragments['gene'] = 'chrM'
    mito_fragments.columns = ['chromosome', 'start', 'end', 'barcode', 'umis', 'gene']
    for row in mito_genes.iterrows():
        row = row[1]
        start = row['start']
        end = row['end']

        mito_fragments.loc[(mito_fragments['start']>=start) & (mito_fragments['end'] < end), 'gene'] = row['gene_name']
    # mito_fragments.head()

    mito_dir = f"{cache_dir}{s}/mito/"
    if not os.path.exists(mito_dir):
        os.mkdir(mito_dir)
        
    # WRITE OUTPUT 
    mito_fragments.to_csv(f'{mito_dir}mito_fragments.csv')

    # Get counts for each gene per cell barcode
    mito_counts = mito_fragments[['barcode', 'gene']].groupby('barcode').value_counts().unstack(fill_value = 0)
    mito_counts.head()
    mito_counts.to_csv(f'{mito_dir}mito_fragments_counts.csv')

## Split frags greater than 3000 bp

From other analysis, we know that frags greater than 3000 bp mostly span the deleted region and are therefore representative of endjoined mito. Thus, we split the fragments into two with a padding of 50. 

For example,

Fragment `7692 - 11869` --> `7692 - 7742` and `11819 - 11869`

If you do not need to remove any mitochondrial fragments, can SKIP this step.

In [27]:
def split_long_frag_df(df, frag_threshold, padding):
    assert padding >= 0, "padding must be positive"
    
    long_frags_df = df[(df['end'] - df['start']) >= frag_threshold]
    start_frags_df =  long_frags_df.copy()
    start_frags_df['end'] = start_frags_df['start'] + padding
    end_frags_df =  long_frags_df.copy()
    end_frags_df['start'] = end_frags_df['end'] - padding
    
    
    return pd.concat([start_frags_df, end_frags_df]).sort_values('start')

In [ ]:
frag_threshold = 3000
padding = 50

for s in samples:
    mito_dir = f"{cache_dir}{s}/mito/"
    
    # READ IN INPUT (output of above code)
    unfilt_path = f'{mito_dir}mito_fragments.csv'
    
    # OUTPUT path
    filt_path = f"{mito_dir}mito_fragments_split_{frag_threshold}_bp_frags.csv"
    
    df = pd.read_csv(unfilt_path, index_col = 0)
    no_long_frags_df = df[(df['end'] - df['start']) < frag_threshold]
    split_frags_df = split_long_frag_df(df, frag_threshold, padding)
    new_frags_df = pd.concat([no_long_frags_df, split_frags_df]).sort_values('start')
    
    new_frags_df.to_csv(filt_path)

# 02 Generate Bulk Mitochondrial Coverage Plots

See `8HetClonesHashtagMix_process_mito_fragments.ipynb`, same code just adapt inputs and outputs. Since not essential and only for visualization purposes, we do not repeat in this notebook. 

# 03 Compute Single-cell coverage (takes longer to run)

For each cell, we report the number of unique fragments (different START and END) that overlap each base pair.

In [ ]:
# relative to repo
cov_script = "../../scripts/single-cell-coverage.py"
mito_genes_path = '../../misc/mito_genes.csv'

samples = [
    "YF-2721_48mix_25dGlu_3d_mito_hLLLdogma_multiome",
    "YF-2721_48mix_10dGal_3d_mito_hLLLdogma_multiome",
    "YF-2721_48mix_25dGlu_NoPyr_3d_mito_hLLLdogma_multiome",
]

for s in samples:
    mito_frags_path = f'{cache_dir}{s}/mito/mito_fragments_split_3000_bp_frags.csv'
    barcodes_path = f"{cache_dir}{s}/mito/barcodes.tsv"
    out_dir = f'{cache_dir}{s}/mito/single-cell-coverage_split_3000_bp_frags/'
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
    
    memory = 10
    conda_env = 'scanpy'
    options = dict()
    options['J'] = f'{s}_coverage'
    options['n'] = f'5'
    options['R'] = f'rusage[mem={memory}]'
    options['W'] = f'120:00'
    options['o'] = f'{s}.stdout'
    options['eo'] = f'{s}.stdout'


    if os.path.exists(options['o']):
        os.remove(options['o'])

    option_string = ' '.join([f'-{flag} {val}' for flag, val in options.items()])
    job_string = f"source ~/.bashrc; conda activate {conda_env};"
    
    # THIS IS THE ACTUAL COMMAND LINE TO RUN THE SCRIPT (everything else is to allow it to run on MSKCC servers)
    run_script_string = f"python {cov_script} --mito_genes_path {mito_genes_path} --mito_frags_path {mito_frags_path} --barcodes_path {barcodes_path} --out_dir {out_dir}"

    command = f'bsub {option_string} \"{job_string} {run_script_string}"'
    os.system(command)
    print(command + '\n')